# 串聯模型鏈 (Sequential Model Chains)

本範例展示如何順序調用多個模型來實現多步驟處理流程。

## 🎯 學習目標
- 理解串聯模型的概念
- 掌握多步驟模型調用的實現
- 學習逐步改進內容的技巧

## 📝 核心概念
串聯模型鏈是將多個模型按順序連接，前一個模型的輸出成為下一個模型的輸入，實現內容的逐步生成和改進。

## ✨ 適用場景
- 內容逐步優化（草稿 → 格式化 → 潤色）
- 多輪對話處理
- 複雜任務分步執行
- 內容品質逐步提升

## 步驟 1：導入套件和建立模型

In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")

print("✅ 模型已建立")

## 步驟 2：定義三個處理階段的 Prompt

我們將郵件回覆流程分為三個階段：
1. **生成階段**：根據客戶郵件生成初步回覆
2. **格式化階段**：將回覆內容格式化
3. **改進階段**：檢查品質並提供改進建議

In [ ]:
# 第一步：生成郵件回覆
reply_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是專業的客服代表。請針對客戶的問題提供簡潔、專業的回覆內容。"),
    ("human", "客戶郵件：\n\n{email_content}")
])

# 第二步：格式化郵件
format_prompt = ChatPromptTemplate.from_messages([
    ("system", "請將以下郵件回覆格式化，加上適當的稱呼、問候語和專業的結尾。"),
    ("human", "郵件內容：\n\n{reply}")
])

# 第三步：品質檢查和改進建議
improve_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
    ("human", "評估並改進這封郵件：\n\n{formatted_reply}")
])

print("✅ 三階段 Prompt 已定義完成")

## 步驟 3：串聯三個模型調用

使用 LCEL 的管道符號 `|` 將三個模型順序連接：
- 每個階段都調用同一個模型，但使用不同的 Prompt
- 使用 lambda 函數將前一步的輸出包裝成下一步需要的格式

In [ ]:
# 串聯三個模型調用
sequential_chain = (
    reply_prompt                                    # 1. 生成回覆 Prompt
    | model                                         # 2. 模型生成回覆
    | StrOutputParser()                             # 3. 解析為字串
    | (lambda x: {"reply": x})                      # 4. 包裝成 dict 傳給下一步
    | format_prompt                                 # 5. 格式化 Prompt
    | model                                         # 6. 模型格式化郵件
    | StrOutputParser()                             # 7. 解析為字串
    | (lambda x: {"formatted_reply": x})            # 8. 包裝成 dict 傳給下一步
    | improve_prompt                                # 9. 改進建議 Prompt
    | model                                         # 10. 模型提供改進建議
    | StrOutputParser()                             # 11. 解析為字串
)

print("✅ 串聯模型鏈已建立完成！")
print("\n流程：生成回覆 → 格式化郵件 → 品質評估與改進建議")

## 步驟 4：測試串聯模型鏈

In [ ]:
# 準備測試資料
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

# 執行串聯鏈
print("="*60)
print("客戶郵件：")
print("="*60)
print(customer_email)

print("\n" + "="*60)
print("串聯模型處理結果（生成 → 格式化 → 品質評估）：")
print("="*60)

result = sequential_chain.invoke({"email_content": customer_email})
print(result)

## 💡 進階範例：可視化每個步驟的輸出

為了更清楚看到每個階段的處理結果，我們可以分別執行每個步驟：

In [ ]:
# 分步執行以展示每個階段的輸出
print("="*60)
print("階段 1️⃣：生成初步回覆")
print("="*60)
step1_chain = reply_prompt | model | StrOutputParser()
step1_result = step1_chain.invoke({"email_content": customer_email})
print(step1_result)

print("\n" + "="*60)
print("階段 2️⃣：格式化郵件")
print("="*60)
step2_chain = format_prompt | model | StrOutputParser()
step2_result = step2_chain.invoke({"reply": step1_result})
print(step2_result)

print("\n" + "="*60)
print("階段 3️⃣：品質評估與改進建議")
print("="*60)
step3_chain = improve_prompt | model | StrOutputParser()
step3_result = step3_chain.invoke({"formatted_reply": step2_result})
print(step3_result)

## 📊 串聯模型鏈總結

### ✅ 優點
- **結構清晰**：每個步驟職責明確，易於理解和維護
- **逐步改進**：內容經過多次處理，品質更高
- **靈活調整**：可以輕鬆增加或修改處理步驟
- **易於除錯**：可以單獨測試每個階段

### ⚠️ 注意事項
- **執行時間較長**：順序執行多個模型，總時間是各步驟之和
- **成本較高**：每次調用都會消耗 API 資源
- **錯誤傳播**：前面步驟的錯誤會影響後續處理

### 🎯 最佳實踐
1. **合理分步**：每個步驟應該有明確的單一職責
2. **控制步驟數**：通常 2-4 個步驟較為理想
3. **添加驗證**：在關鍵步驟之間加入結果驗證
4. **考慮效能**：評估是否真的需要多次模型調用

### 🔗 相關技術
- [並行模型鏈](8_chains_parallel_model_ollama.ipynb)：同時調用多個模型
- [閉包模型鏈](7_chains_closure_model_ollama.ipynb)：在 Lambda 中靈活控制模型調用
- [動態提示鏈](9_chains_dynamic_prompt_ollama.ipynb)：根據內容動態調整處理邏輯